# 함수 호출

# 개요
OpenAI의 함수 호출 기능은 AI가 생성한 텍스트를 외부 함수와 통합할 수 있도록 합니다. \
이를 통해 모델의 출력에 따라 코드를 실행하거나 계산을 수행하거나 데이터를 가져올 수 있습니다. \
함수를 정의하고 모델이 이를 호출하도록 허용함으로써 더 상호작용적이고 동적인 애플리케이션을 만들 수 있습니다.

더 자세한 예제는 공식 문서를 참조하세요: [Azure OpenAI 서비스의 함수 호출](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling?tabs=non-streaming%2Cpython)



## 함수 호출의 필요성

함수 호출 이전에는 LLM의 응답이 비구조적이고 일관성이 없었습니다.  
개발자는 각 응답의 변형을 처리할 수 있도록 복잡한 검증 코드를 작성해야 했습니다.  
사용자는 "스톡홀름의 현재 날씨는 무엇인가요?"와 같은 질문에 답을 받을 수 없었습니다.  
이는 모델이 학습된 데이터의 시간에 제한되었기 때문입니다.

함수 호출은 Azure OpenAI 서비스의 기능으로 다음과 같은 제한을 극복할 수 있습니다:

- **일관된 응답 형식**. 응답 형식을 더 잘 제어할 수 있다면 응답을 다른 시스템에 통합하기가 더 쉬워집니다.  
- **외부 데이터**. 애플리케이션의 다른 소스 데이터를 채팅 컨텍스트에서 사용할 수 있는 기능.  

함수 호출을 사용하면 구조화된 데이터를 받을 수 있습니다.  
함수 호출을 사용할 때 LLM이 실제로 함수 호출을 실행하거나 실행하지는 않습니다. 대신, 우리는 LLM이 응답을 따를 구조를 만듭니다.  
그런 다음 이러한 구조화된 응답을 사용하여 애플리케이션에서 실행할 함수를 결정합니다.  

![function flow](image/Function-Flow.png)

그런 다음 함수에서 반환된 데이터를 가져와 LLM에 다시 보냅니다. LLM은 자연어를 사용하여 사용자의 질문에 답변합니다.

## 함수 호출 생성하기

함수 호출을 생성하는 과정은 다음의 3단계로 이루어집니다:

1. **사용자 메시지 + 함수 목록을 전달하며 Chat Completions API를 호출합니다.**  
   모델은 사용자의 질문을 보고, 준비된 함수 중 어떤 것이 필요한지 판단합니다.  
   이 단계에서는 LLM이 "실제로 함수 실행을 하지는 않고",  
   **어떤 함수를 어떤 인자로 호출해야 하는지 구조화된 형태(JSON)로 알려줍니다.**

2. **모델이 선택한 함수 호출 정보를 읽고 실제로 함수를 실행합니다.**  
   애플리케이션(서버/백엔드)이 그 JSON을 해석하여  
   API 호출, DB조회, 날씨 API 요청 등 **실제 동작을 수행합니다.**  
   즉, 이 단계는 “LLM이 요청한 작업을 실제 시스템이 실행하는 단계”입니다.

3. **함수 실행 결과를 다시 Chat Completions API에 전달하여 최종 답변을 생성합니다.**  
   함수 호출 결과(예: 날씨 정보, DB 결과, 검색 결과 등)를 LLM에게 다시 전달하면  
   LLM은 그 데이터를 기반으로 **자연어로 된 최종 응답을 만들고 사용자에게 보내는 단계**입니다.

![LLM Flow](image/LLM-Flow.png)


In [2]:
import os
from openai import AzureOpenAI
import json
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-12-01-preview"
)
CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

---

## 날씨 조회 함수

실시간 날씨 정보를 조회하는 함수를 만들어보겠습니다.

In [3]:
import requests

# 날씨 조회 함수 (wttr.in API 사용)
def get_current_weather(location, unit="celsius"):
    """
    Fetch current weather using the wttr.in public API (no API key required).
    """
    # wttr.in supports city names directly in the URL
    # unit: 'm' for metric (Celsius), 'u' for US (Fahrenheit)
    unit_flag = 'u' if unit == "fahrenheit" else 'm'
    url = f"https://wttr.in/{location}?format=j1&{unit_flag}"
    try:
        resp = requests.get(url, timeout=5)
        resp.raise_for_status()
        data = resp.json()
        # wttr.in returns temperature in Celsius by default
        temp_c = data["current_condition"][0]["temp_C"]
        temp_f = data["current_condition"][0]["temp_F"]
        city = location
        temperature = temp_f if unit == "fahrenheit" else temp_c
        return json.dumps({"location": city, "temperature": temperature, "unit": unit})
    except Exception as e:
        return json.dumps({"location": location, "temperature": "unknown", "error": str(e)})



## 날씨 조회 실행

In [4]:
def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role":"system", "content":"""귀하는 사용자가 날씨에 대한 정보를 얻을 수 있도록 설계된 AI 어시스턴트입니다. 
                 사용자가 날씨에 대한 정보를 요청하면 'get_current_weather'라는 도구 함수를 사용해야 합니다. 
                 제공된 함수만 사용하세요.
                 함수에 어떤 값을 입력할지 함부로 추측하지 마세요. 
                 사용자 요청이 모호한 경우 설명을 요청하세요.
                 인수의 형식은 '/*'를 포함해서는 안 되며, 속성 이름이 큰따옴표로 묶인 유효한 JSON 문자열이어야 합니다."""},
                 {"role": "user", "content": "서울 상암의 날씨는 어떤가요?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=messages,
        tools=tools,
        tool_choice = {"type": "function", "function": {"name": "get_current_weather"}}
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    # 디버깅: Tool Call 확인
    print("="*60)
    if tool_calls:
        print("✅ LLM이 함수 호출을 결정했습니다!")
        for tool_call in tool_calls:
            print(f" 호출할 함수: {tool_call.function.name}")
            print(f" 전달할 인자: {tool_call.function.arguments}")
    else:
        print("❌ 함수 호출 없이 직접 답변합니다.")
    print("="*60)
    
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            # Sometimes the name is classified mistakenly as python
            if function_name == "python":
                function_name = "get_current_weather"
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            print(f"🌐 API 응답: {function_response}")  # 실제 API 결과
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print(f"💬 최종 답변: {second_response.choices[0].message.content}")
        print("="*60)
        return second_response
    


print(run_conversation().choices[0].message.content)

✅ LLM이 함수 호출을 결정했습니다!
 호출할 함수: get_current_weather
 전달할 인자: {"location":"서울 상암"}
🌐 API 응답: {"location": "\uc11c\uc6b8 \uc0c1\uc554", "temperature": "2", "unit": null}
💬 최종 답변: 서울 상암의 현재 온도는 2도입니다. 추가로 필요한 날씨 정보(예: 강수, 풍속 등)가 있으시면 말씀해 주세요.
서울 상암의 현재 온도는 2도입니다. 추가로 필요한 날씨 정보(예: 강수, 풍속 등)가 있으시면 말씀해 주세요.


---

## 예제 2: 환율 조회 함수

이번에는 실시간 환율 정보를 조회하는 함수를 만들어보겠습니다.

In [5]:
# 환율 조회 함수 (ExchangeRate-API 사용)
def get_exchange_rate(from_currency, to_currency):
    """
    Fetch current exchange rate using the ExchangeRate-API (no API key required).
    """
    url = f"https://open.er-api.com/v6/latest/{from_currency}"
    try:
        resp = requests.get(url, timeout=5)
        resp.raise_for_status()
        data = resp.json()
        
        if to_currency in data["rates"]:
            rate = data["rates"][to_currency]
            return json.dumps({
                "from": from_currency,
                "to": to_currency,
                "rate": rate,
                "last_update": data["time_last_update_utc"]
            })
        else:
            return json.dumps({"error": f"Currency {to_currency} not found"})
    except Exception as e:
        return json.dumps({"error": str(e)})

In [6]:
def run_exchange_query():
    messages = [
        {"role": "system", "content": """당신은 환율 정보를 제공하는 AI 어시스턴트입니다.
        사용자가 환율에 대한 정보를 요청하면 'get_exchange_rate' 함수를 사용해야 합니다.
        제공된 함수만 사용하세요."""},
        {"role": "user", "content": "오늘 1달러는 한국 돈으로 얼마인가요?"}
    ]
    
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_exchange_rate",
                "description": "Get the current exchange rate between two currencies",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "from_currency": {
                            "type": "string",
                            "description": "The source currency code, e.g. USD, EUR, KRW",
                        },
                        "to_currency": {
                            "type": "string",
                            "description": "The target currency code, e.g. USD, EUR, KRW",
                        },
                    },
                    "required": ["from_currency", "to_currency"],
                },
            },
        }
    ]
    
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    # 디버깅: Tool Call 확인
    print("="*60)
    if tool_calls:
        print("✅ LLM이 함수 호출을 결정했습니다!")
        for tool_call in tool_calls:
            print(f" 호출할 함수: {tool_call.function.name}")
            print(f" 전달할 인자: {tool_call.function.arguments}")
    else:
        print("❌ 함수 호출 없이 직접 답변합니다.")
    print("="*60)
    
    if tool_calls:
        available_functions = {
            "get_exchange_rate": get_exchange_rate,
        }
        messages.append(response_message)
        
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                from_currency=function_args.get("from_currency"),
                to_currency=function_args.get("to_currency"),
            )
            print(f"🌐 API 응답: {function_response}")  # 실제 API 결과
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
        
        second_response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=messages,
        )
        print(f"💬 최종 답변: {second_response.choices[0].message.content}")
        print("="*60)
        return second_response

print(run_exchange_query().choices[0].message.content)

✅ LLM이 함수 호출을 결정했습니다!
 호출할 함수: get_exchange_rate
 전달할 인자: {"from_currency":"USD","to_currency":"KRW"}
🌐 API 응답: {"from": "USD", "to": "KRW", "rate": 1453.518498, "last_update": "Mon, 17 Nov 2025 00:02:32 +0000"}
💬 최종 답변: 오늘 1달러(USD)는 약 1,453.52원(KRW)입니다. 환율은 변동될 수 있으니 참고하시기 바랍니다.
오늘 1달러(USD)는 약 1,453.52원(KRW)입니다. 환율은 변동될 수 있으니 참고하시기 바랍니다.


#### 탐색할 사용 사례
1. **동적 계산**  \
사용자 입력에 따라 실시간으로 계산을 수행합니다.

2. **데이터 가져오기** \
모델의 출력에 따라 외부 API에서 데이터를 가져옵니다.

3. **상호작용 애플리케이션** \
AI와 백엔드 함수를 통합하여 더 상호작용적이고 동적인 애플리케이션을 만듭니다.

#### 모범 사례
1. **명확한 프롬프트** \
모델로부터 정확한 응답을 얻기 위해 프롬프트를 명확하고 구체적으로 작성하세요.

2. **견고한 파싱** \
모델 출력의 다양한 형식을 처리할 수 있도록 견고한 파싱 로직을 구현하세요.

3. **오류 처리** \
예상치 못한 출력이나 함수 호출 실패를 관리하기 위해 오류 처리를 포함하세요.